In [ ]:
import polars as pl
# 列表示を省略しない
pl.Config.set_tbl_cols(-1)
# 行の表示を省略しない
pl.Config.set_tbl_rows(-1)

# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [ ]:
uriage_data = pl.read_csv("uriage.csv")
uriage_data.head()

In [ ]:
kokyaku_data = pl.read_excel("kokyaku_daicho.xlsx")
kokyaku_data.head()

### ノック１２：データの揺れを見てみよう

In [ ]:
uriage_data["item_name"].head()

In [ ]:
uriage_data["item_price"].head()

### ノック１３：データに揺れがあるまま集計しよう

In [ ]:
uriage_data = (
    uriage_data.with_columns(
        uriage_data["purchase_date"].str.to_datetime().alias("purchase_date")
    )
)
uriage_data = (
    uriage_data.with_columns([
        uriage_data["purchase_date"].dt.strftime("%Y%m").alias("purchase_month")
    ])
)

res = uriage_data.pivot(index = "purchase_month", columns = "item_name", values = "item_name", aggregate_function = "count").sort("purchase_month")
res

In [ ]:
res = uriage_data.pivot(index = "purchase_month", columns = "item_name", values = "item_price", aggregate_function = "sum").sort("purchase_month")
res

### ノック１４：商品名の揺れを補正しよう

In [ ]:
print( len(uriage_data["item_name"].unique()) )

In [ ]:
uriage_data = (
    uriage_data.with_columns(
        uriage_data["item_name"]
        .str.to_uppercase()
        .str.replace_all("　", "")
        .str.replace_all(" ", "")
        .alias("item_name")
    )
)

uriage_data.sort(by = ["item_name"]).head()

In [ ]:
print( uriage_data["item_name"].unique() )
print( len(uriage_data["item_name"].unique()) )

### ノック１５：金額欠損値の補完をしよう

In [ ]:
uriage_data.null_count()

In [ ]:
# 元のコードでは要素ごとに商品価格の最大値を検索していた
# 毎回検索するのはもったいないので、あらかじめ商品名と商品価格の辞書を作成しておく
item_name_price = (
    uriage_data[["item_name", "item_price"]]
    .drop_nulls()
    .sort(by = "item_price", descending = True)
    .unique(subset = "item_name", keep = "first")
    .sort(by = "item_name", descending = False)
)

dict_name_price = {}
for name, price in zip(item_name_price["item_name"], item_name_price["item_price"]):
    dict_name_price[name] = price

# 商品価格の欠損値を埋める
uriage_data = (
    uriage_data.with_columns(
        uriage_data["item_name"]
        .map_elements(lambda name: dict_name_price[name])
        .alias("item_price")
    )
)

uriage_data.head()

In [ ]:
uriage_data.null_count()

In [ ]:
for name in uriage_data["item_name"].unique().sort():
    print(name
          + "の最大値:" + str(uriage_data.filter(pl.col("item_name") == name)["item_price"].max())
          + "\tの最小値:" + str(uriage_data.filter(pl.col("item_name") == name)["item_price"].min()))

### ノック１６：顧客名の揺れを補正しよう

In [ ]:
kokyaku_data["顧客名"].head()

In [ ]:
uriage_data["customer_name"].head()

In [ ]:
kokyaku_data = (
    kokyaku_data.with_columns(
        kokyaku_data["顧客名"]
        .str.replace_all("　", "")
        .str.replace_all(" ", "")
        .alias("顧客名")
    )
)
kokyaku_data["顧客名"].head(n = 5)

### ノック１７：日付の揺れを補正しよう

In [ ]:
flg_is_serial = kokyaku_data["登録日"].cast(pl.Int64, strict = False)
len( flg_is_serial.drop_nulls() )

In [ ]:
from datetime import datetime, timedelta

# Excelの日付形式をPythonのdatetime型に変換する関数を定義します。
def convert_excel_date_to_datetime(excel_date):
    return datetime(1899, 12, 30) + timedelta(days=int(excel_date))

# 日付と数字が混在したSeriesを、Excel書式の日付にに変換する
converted_date = (
    kokyaku_data["登録日"].cast(pl.Utf8)
    .map_elements(lambda x: convert_excel_date_to_datetime(x)
                  if x.isdigit() else datetime.strptime(x, "%Y/%m/%d"), return_dtype = pl.Date)
)

converted_date.head(n = 5)

kokyaku_data = (
    kokyaku_data.with_columns(
        converted_date.alias("登録日")
    )
)

In [ ]:
kokyaku_data = (
    kokyaku_data.with_columns(
        converted_date.dt.strftime("%Y%m").alias("登録年月")
    )
)

rslt = kokyaku_data.group_by(by = "登録年月").agg( pl.col("顧客名").count() ).sort(by = "登録年月")
print(rslt)
print(len(kokyaku_data))

In [ ]:
flg_is_serial = kokyaku_data["登録日"].cast(pl.Int64, strict = False)
len( flg_is_serial.drop_nulls() )

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [ ]:
# 売上実績のcustomer_name列と、顧客名簿の顧客名列とをキーにして結合する
# 中間データを顧客名で保存するので、結合時にcustomer_name列の列名を変更する
join_data = (
    uriage_data
    .rename(mapping = {"customer_name": "顧客名"})
    .join(other = kokyaku_data, on="顧客名", how="left")
)
join_data.head()

### ノック１９：クレンジングしたデータをダンプしよう

In [ ]:
dump_data = join_data[["purchase_date", "purchase_month", "item_name", "item_price", "顧客名", "かな", "地域", "メールアドレス", "登録日"]]
dump_data.head()

In [ ]:
dump_data.write_csv("dump_data.csv")

### ノック２０：データを集計しよう

In [ ]:
import_data = pl.read_csv("dump_data.csv")
import_data.head()

In [ ]:
byItem = (
    import_data
    .pivot(index = "purchase_month",
           columns = "item_name",
           values = "item_name",
           aggregate_function = "count",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byItem

In [ ]:
byPrice = (
    import_data
    .pivot(index = "purchase_month",
           columns = "item_name",
           values = "item_price",
           aggregate_function = "sum",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byPrice

In [ ]:
byCustomer = (
    import_data
    .pivot(index = "purchase_month",
           columns = "顧客名",
           values = "顧客名",
           aggregate_function = "count",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byCustomer

In [ ]:
byRegion = (
    import_data
    .pivot(index = "purchase_month",
           columns = "地域",
           values = "地域",
           aggregate_function = "count",
           sort_columns = True)
           .sort(by = "purchase_month")
)
byRegion

In [ ]:
away_data = (
    uriage_data
    .rename(mapping = {"customer_name": "顧客名"})
    .join(other = kokyaku_data, on="顧客名", how="outer")
)
away_data.filter( pl.col("purchase_date").is_null() )[["顧客名", "メールアドレス", "登録日"]]

#away_data.filter( pl.col("顧客名") == "福井美希")